In [23]:
import tensorflow as tf
import pygame
import scipy.io as sio
from scipy import misc
import numpy as np

In [24]:
def weights(shape):
    return tf.Variable(tf.random_normal(shape))

def bias(shape):
    return tf.Variable(tf.random_normal(shape))

In [52]:

def neural_network_model(data):
    #input data * weight) + biases
    total_nodes_hl1=256
    total_nodes_hl2=256
    total_classes=10
    hidden_layer_1 = {'weights':weights([900,total_nodes_hl1]),'biases':bias([total_nodes_hl1]) }
    hidden_layer2 ={'weights':weights([total_nodes_hl1,total_nodes_hl2]),'biases':bias([total_nodes_hl2])}

    output_layer = {'weights':weights([total_nodes_hl1,total_classes]),'biases':bias([total_classes])}

    layer_1 = tf.add(tf.matmul(data,hidden_layer_1['weights']), hidden_layer_1['biases'])
	#rectified linear as threshold function
    layer_1 = tf.nn.relu(layer_1)
    output = tf.matmul(layer_1,output_layer['weights']) + output_layer['biases']
    return output



In [88]:
def convertToOneHot(trainArray):
    a=np.array(trainArray,'int32')
    d=tf.stack(a)
    return d



In [93]:
inputTheta = sio.loadmat('newX.mat')
x=inputTheta['X']
epoch_x = tf.placeholder('float32',[None,x.shape[1]])
epoch_test_x = tf.placeholder('float32',[x.shape[1],None])
epoch_y = tf.placeholder('float32',[None,10])
y= np.array(sio.loadmat('newY.mat')['y'])
xTrain,xTest,yTrain,yTest=splitData(x,y)
print(xTest.shape

In [94]:
def splitData(X, y):
    """Split sample data into training set (80%) and testing set (20%)"""

    size1 =int( X.shape[0] * 0.8)
    size2 = int(X.shape[0] * 0.2)
    Xtrain = np.zeros((size1,X.shape[1]))
    Xtest = np.zeros((size2+1,X.shape[1]))
    ytrain = np.zeros((size1,1))
    ytest = np.zeros((size2+1,1))

    for i, v in enumerate(np.random.permutation(len(y))):
        #print(i, y[v], len(X[v]))

        try:
            Xtrain[i] = X[v]
            ytrain[i] = y[v]
        except:
            Xtest[i-size1] = X[v]
            ytest[i-size1] = y[v]
    return Xtrain, Xtest, ytrain, ytest

In [101]:
def train_neural_net(xTrain):     
    oneHotYTrain=tf.one_hot(convertToOneHot(yTrain),10,1.0,0.0)
    oneHotYTest=tf.one_hot(convertToOneHot(yTest),10,1.0,0.0)
    prediction = neural_network_model(epoch_x)    
    oneHotYTrain=tf.reshape(oneHotYTrain,[2115,10])    
    oneHotYTest=tf.reshape(oneHotYTest,[529,10])   

    print(oneHotYTrain)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=epoch_y))

    optimizer = tf.train.AdamOptimizer().minimize(cost)

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        hm_epochs = 15

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(y.shape[0]/100)):

                _, c = sess.run([optimizer, cost], feed_dict= {epoch_x:xTrain, epoch_y:oneHotYTrain.eval()})
                epoch_loss += c
            print('Epoch', epoch, 'UNDERSCORE', _,'loss:',epoch_loss)
        print(prediction.get_shape())
        print(oneHotYTrain.get_shape())
        

        correcct = tf.equal(tf.argmax(prediction,1),tf.argmax(epoch_y,1))
        
        accuracy = tf.reduce_mean(tf.cast(correcct,'float'))
        print(accuracy)
        print('Accuracy:', accuracy.eval({epoch_x:xTest,epoch_y:oneHotYTest.eval()}))
        print(xTest.shape,'-----',oneHotYTest.get_shape(),'--------',prediction,'=====0',yTest.shape)
        prediction=tf.argmax(prediction,1)
        final_result=prediction.eval({epoch_test:xTest})
        print (prediction.eval({epoch_test_x:xTest}))
        # classification =sess.run(prediction)
        print(yTest.transpose())
        print(tf.reduce_mean(tf.cast(tf.equal(yTest,final_result).eval(),'float')).eval())

In [102]:
if __name__ == "__main__":
    train_neural_net(xTrain)

Tensor("Reshape_100:0", shape=(2115, 10), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 UNDERSCORE None loss: 1873.56634521
Epoch 1 UNDERSCORE None loss: 566.19137001
Epoch 2 UNDERSCORE None loss: 224.31885004
Epoch 3 UNDERSCORE None loss: 107.292384624
Epoch 4 UNDERSCORE None loss: 53.964746356
Epoch 5 UNDERSCORE None loss: 26.4534141421
Epoch 6 UNDERSCORE None loss: 12.2565356195
Epoch 7 UNDERSCORE None loss: 5.35604602098
Epoch 8 UNDERSCORE None loss: 2.65198022872
Epoch 9 UNDERSCORE None loss: 1.35606110841
Epoch 10 UNDERSCORE None loss: 0.686357814819
Epoch 11 UNDERSCORE None loss: 0.299015014432
Epoch 12 UNDERSCORE None loss: 0.0934687543195
Epoch 13 UNDERSCORE None loss: 0.0192611565581
Epoch 14 UNDERSCORE None loss: 0.0128459086409
(?, 10)
(2115, 10)
Tensor("Mean_47:0", shape=(), dtype=float32)
Accuracy: 0.795841
(529, 900) ----- (529, 10) -------- Tensor("add_20:0", shape=(?, 10), dtype=float32) =====0 (529, 1)


NameError: name 'epoch_test' is not defined

In [ ]:
def create_screen():
    """Main method. Draw interface"""
    
    global screen
    pygame.init()
    screen = pygame.display.set_mode((730, 450))
    pygame.display.set_caption("Handwriting recognition")
    
    background = pygame.Surface((360,360))
    background.fill((255, 255, 255))
    background2 = pygame.Surface((360,360))
    background2.fill((255, 255, 255))
    
    clock = pygame.time.Clock()
    keepGoing = True
    lineStart = (0, 0)
    drawColor = (255, 0, 0)
    lineWidth = 15
    
    inputTheta = sio.loadmat('scaledTheta.mat')
    theta = inputTheta['t']
    num_hidden = 25
    num_input = 900
    num_lables = 10

    Theta1 = np.reshape(theta[:num_hidden*(num_input+1)], (num_hidden,-1))
    Theta2 = np.reshape(theta[num_hidden*(num_input+1):], (num_lables,-1))

    pygame.display.update()
    image = None
            
    while keepGoing:
        
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                keepGoing = False
            elif event.type == pygame.MOUSEMOTION:
                lineEnd = pygame.mouse.get_pos()
                if pygame.mouse.get_pressed() == (1, 0, 0):
                    pygame.draw.line(background, drawColor, lineStart, lineEnd, lineWidth)
                lineStart = lineEnd
            elif event.type == pygame.MOUSEBUTTONUP:
                screen.fill((0, 0, 0))
                screen.blit(background2, (370, 0))
                #w = threading.Thread(name='worker', target=worker)
                image = calculateImage(background, screen, Theta1, Theta2, lineWidth)

            elif event.type == pygame.KEYDOWN:
                myData = (event, background, drawColor, lineWidth, keepGoing, screen, image)
                myData = checkKeys(myData)
                (event, background, drawColor, lineWidth, keepGoing) = myData
        
        
        screen.blit(background, (0, 0))
        pygame.display.flip()

In [65]:
def calculateImage(background, screen, Theta1, Theta2, lineWidth):
    """Crop and resize the input"""
    
    global changed
    focusSurface = pygame.surfarray.array3d(background)
    focus = abs(1-focusSurface/255)
    focus = np.mean(focus, 2) 
    x = []
    xaxis = np.sum(focus, axis=1)
    for i, v in enumerate(xaxis):
        if v > 0:
            x.append(i)
            break
    for i, v in enumerate(xaxis[ : :-1]):
        if v > 0:
            x.append(len(xaxis)-i)
            break
    
    y = []
    yaxis = np.sum(focus, axis=0)
    for i, v in enumerate(yaxis):
        if v > 0:
            y.append(i)
            break
    for i, v in enumerate(yaxis[ : :-1]):
        if v > 0:
            y.append(len(yaxis)-i)
            break

    try:
        dx = x[1]-x[0]
        dy = y[1]-y[0]
        bound = focus.shape[0]      
        if dx > dy:
            d = dx-dy
            y0t = y[0] - d//2
            y1t = y[1] + d//2+d%2
            if y0t < 0: y0t = y[0]; y1t = y[1] + d
            if y1t > bound: y0t = y[0] - d; y1t = y[1]
            y[0], y[1] = y0t, y1t
        else:
            d = dy-dx
            x0t = x[0] - d//2
            x1t = x[1] + d//2+d%2
            if x0t < 0: x0t = x[0]; x1t = x[1] + d
            if x1t > bound: x0t = x[0] - d; x1t = x[1]
            x[0], x[1] = x0t, x1t 
        dx = x[1]-x[0]
        dy = y[1]-y[0]
        changed = True
        crop_surf =  pygame.Surface((dx,dy))
        crop_surf.blit(background,(0,0),(x[0],y[0],x[1],y[1]), special_flags=BLEND_RGBA_MAX)
        scaledBackground = pygame.transform.smoothscale(crop_surf, (30, 30))
            
        image = pygame.surfarray.array3d(scaledBackground)
        image = abs(1-image/253)
        image = np.mean(image, 2) 
        image = np.matrix(image.ravel())
        drawPixelated(image, screen)
        (value, prob), (value2, prob2) = probabilty(Theta1,Theta2,image)
        prob = round(prob,1)            
        myLabel = showStats(lineWidth, value, prob)
        (x,y) = screen.get_size()
        screen.blit(myLabel, (17, y-90))
        screen.blit(myLabelSmall, (20, y-38))
    except:
        image = np.zeros((30,30))

    return image

In [66]:
def showStats(lineWidth, value, prob):
    """ shows the current statistics """
    
    myFont = pygame.font.SysFont("Verdana", 50)
    stats = "Estimate: %s    P: %s" % (value, prob)
    statSurf = myFont.render(stats+"%", 1, ((255, 255, 255)))
    return statSurf